In [13]:
import numpy as np



# Example 2D NumPy array
arr = np.array([[1, 2, 3],
                [4, 5, 6],
                [7, 8, 9]])

# Get the shape of the array
original_shape = arr.shape

# Pad out the array with the final column
padded_arr = np.pad(arr, ((0, 2), (0, 0)), mode='constant')

print(original_shape[0])
print(padded_arr.shape[0])

print("Original Array:")
print(arr)
print("Padded Array:")
print(padded_arr)

for i, action in enumerate(arr):
    print(action)
    print(padded_arr[i,:])

3
5
Original Array:
[[1 2 3]
 [4 5 6]
 [7 8 9]]
Padded Array:
[[1 2 3]
 [4 5 6]
 [7 8 9]
 [0 0 0]
 [0 0 0]]
[1 2 3]
[1 2 3]
[4 5 6]
[4 5 6]
[7 8 9]
[7 8 9]


In [ ]:
print(padded_arr[:3,...])
for i, j in zip([1,2,3], [4,5,6]):
    print(i,j)

In [ ]:
esp32 = "http://192.168.1.182"

import requests



In [1]:
import os
import numpy as np
import cv2
import h5py
import argparse
import time
from visualize_episodes import visualize_joints, visualize_timestamp, save_videos

import matplotlib.pyplot as plt
from constants import DT

import IPython
e = IPython.embed

JOINT_NAMES = ["waist", "shoulder", "elbow", "forearm_roll", "wrist_angle", "wrist_rotate"]
STATE_NAMES = JOINT_NAMES + ["gripper"]

MIRROR_STATE_MULTIPLY = np.array([-1, 1, 1, -1, 1, -1, 1]).astype('float32')
MIRROR_BASE_MULTIPLY = np.array([1, -1]).astype('float32')

def load_hdf5(dataset_dir, dataset_name):
    dataset_path = os.path.join(dataset_dir, dataset_name + '.hdf5')
    if not os.path.isfile(dataset_path):
        print(f'Dataset does not exist at \n{dataset_path}\n')
        exit()

    with h5py.File(dataset_path, 'r') as root:
        is_sim = root.attrs['sim']
        compressed = root.attrs.get('compress', False)
        qpos = root['/observations/qpos'][()]
        # qvel = root['/observations/qvel'][()]
        action = root['/action'][()]
        image_dict = dict()
        for cam_name in root[f'/observations/images/'].keys():
            image_dict[cam_name] = root[f'/observations/images/{cam_name}'][()]
        if 'base_action' in root.keys():
            print('base_action exists')
            base_action = root['/base_action'][()]
        else:
            base_action = None
        if compressed:
            compress_len = root['/compress_len'][()]

    if compressed:
        for cam_id, cam_name in enumerate(image_dict.keys()):
            # un-pad and uncompress
            padded_compressed_image_list = image_dict[cam_name]
            image_list = []
            for padded_compressed_image in padded_compressed_image_list: # [:1000] to save memory
                image = cv2.imdecode(padded_compressed_image, 1)
                image_list.append(image)
            image_dict[cam_name] = np.array(image_list)

    return qpos, action, base_action, image_dict, is_sim

In [2]:
data_dir = "/home/jonny/projects/mobile_aloha_world/episodes/task1"
dataset_name = 'episode_1'

In [3]:
qpos,  action, base_action, image_dict, is_sim = load_hdf5(data_dir, dataset_name)

        # process proprioception
qpos = np.array(qpos).astype('int32') #np.concatenate([qpos[:, 7:] * MIRROR_STATE_MULTIPLY, qpos[:, :7] * MIRROR_STATE_MULTIPLY], axis=1)
# qvel = np.concatenate([qvel[:, 7:] * MIRROR_STATE_MULTIPLY, qvel[:, :7] * MIRROR_STATE_MULTIPLY], axis=1)
action = np.array(action).astype('int32')  #np.concatenate([action[:, 7:] * MIRROR_STATE_MULTIPLY, action[:, :7] * MIRROR_STATE_MULTIPLY], axis=1)
if base_action is not None:
    base_action = np.array(base_action).astype('int32') #* MIRROR_BASE_MULTIPLY
    while (base_action.shape[0] < action.shape[0]):
        #pad out array
        base_action=np.pad(base_action, ((0, 1), (0, 0)), mode='constant')

base_action exists


In [12]:
base_action.T.shape[0]

4

In [16]:
import numpy as np

# Assuming 'array' is your 4x150 array
# array = [[n1_1, n1_2, ..., n1_150],
#          [n2_1, n2_2, ..., n2_150],
#          [n3_1, n3_2, ..., n3_150],
#          [n4_1, n4_2, ..., n4_150]]

def process_array(array):
    array = array.T
    result = np.zeros((2, array.shape[1])).astype('int32')  # Initialize result array
    
    # Extracting n1, n2, n3, and n4
    n1 = array[0]
    n2 = array[1]
    n3 = array[2]
    n4 = array[3]
    
    # Conditions
    result[0] = np.where(n3 == 0, n1, np.where((n3 == 1) & (n4 == 1), 0, -n1))
    result[1] = np.where(n3 == 0, n2, np.where((n3 == 1) & (n4 == 1), 0, -n2))
    
    return result.T
new_array =process_array(base_action)
# Example usage:
# array = np.random.randint(10, size=(4, 150))  # Example random array
# result_array = process_array(array)


In [23]:
import torch
all_action_data = []
all_action_data.append(torch.from_numpy(new_array.astype('float')))
all_action_data.append(torch.from_numpy(new_array.astype('float')))
all_action_data = torch.cat(all_action_data, dim=0)
all_action_data.mean(dim=[0]).float()

tensor([-0.0959, -0.1301])

In [ ]:
#change from rgb to bgr
image_dict['top'] = image_dict['top'][:, :, ::-1]
image_dict['base'] = image_dict['base'][:, :, ::-1]


In [7]:
data_dict = {
    '/observations/qpos': qpos,
    # '/observations/qvel': qvel,
    '/action': action,
    '/base_action': base_action,
} if base_action is not None else {
    '/observations/qpos': qpos,
    # '/observations/qvel': qvel,
    '/action': action,
}
for cam_name in image_dict.keys():
    if 'base' in image_dict.keys():
        data_dict[f'/observations/images/{cam_name}'] = image_dict[cam_name]
max_timesteps = len(qpos)

COMPRESS = True


In [8]:
t0 = time.time()
encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), 50] # tried as low as 20, seems fine
compressed_len = []
for cam_name in image_dict.keys():
    image_list = data_dict[f'/observations/images/{cam_name}']
    print(len(image_list))
    compressed_list = []
    compressed_len.append([])
    for image in image_list:
        result, encoded_image = cv2.imencode('.jpg', image, encode_param) # 0.02 sec # cv2.imdecode(encoded_image, 1)
        compressed_list.append(encoded_image)
        compressed_len[-1].append(len(encoded_image))
    data_dict[f'/observations/images/{cam_name}'] = compressed_list
print(f'compression: {time.time() - t0:.2f}s')

# pad so it has same length
t0 = time.time()


146
146
compression: 0.75s


In [9]:
print(compressed_len)

[[3559, 3575, 3587, 3562, 3560, 3449, 3563, 3730, 3580, 3716, 3843, 3663, 3734, 3881, 3870, 3863, 3889, 3821, 3852, 3870, 3880, 3866, 3907, 3927, 3895, 3955, 3900, 3853, 3867, 3912, 3841, 3842, 3893, 4069, 4065, 4327, 4382, 4373, 4404, 4551, 4553, 4564, 4538, 4417, 4329, 4338, 4330, 4320, 4300, 4237, 4251, 5159, 5216, 5258, 5275, 5235, 5321, 5760, 5791, 5632, 5694, 5146, 4678, 4341, 3747, 3233, 3248, 3188, 3247, 3280, 3270, 3172, 3245, 3170, 3268, 3165, 3276, 3279, 3216, 3271, 3279, 3284, 3223, 4645, 5854, 5972, 5817, 5792, 5768, 5074, 4864, 4478, 4841, 4727, 4998, 4989, 5009, 4846, 4587, 4833, 4984, 4832, 5053, 5100, 5080, 5165, 5188, 5160, 4913, 4914, 5032, 5037, 5005, 5146, 5182, 4874, 4692, 4536, 4166, 4126, 4151, 4034, 4132, 4030, 4037, 4004, 4103, 4125, 4142, 4109, 4109, 3845, 3791, 3858, 3894, 3701, 3798, 3802, 3840, 3817, 3844, 3823, 3816, 3783, 3796, 3821], [29907, 29890, 29907, 29859, 29913, 30291, 29145, 29834, 29216, 28855, 28983, 28852, 28813, 29018, 28442, 28786, 28755, 2

In [10]:
compressed_len = np.array(compressed_len)
print(compressed_len)
padded_size = compressed_len.max()
print(padded_size)

[[ 3559  3575  3587  3562  3560  3449  3563  3730  3580  3716  3843  3663
   3734  3881  3870  3863  3889  3821  3852  3870  3880  3866  3907  3927
   3895  3955  3900  3853  3867  3912  3841  3842  3893  4069  4065  4327
   4382  4373  4404  4551  4553  4564  4538  4417  4329  4338  4330  4320
   4300  4237  4251  5159  5216  5258  5275  5235  5321  5760  5791  5632
   5694  5146  4678  4341  3747  3233  3248  3188  3247  3280  3270  3172
   3245  3170  3268  3165  3276  3279  3216  3271  3279  3284  3223  4645
   5854  5972  5817  5792  5768  5074  4864  4478  4841  4727  4998  4989
   5009  4846  4587  4833  4984  4832  5053  5100  5080  5165  5188  5160
   4913  4914  5032  5037  5005  5146  5182  4874  4692  4536  4166  4126
   4151  4034  4132  4030  4037  4004  4103  4125  4142  4109  4109  3845
   3791  3858  3894  3701  3798  3802  3840  3817  3844  3823  3816  3783
   3796  3821]
 [29907 29890 29907 29859 29913 30291 29145 29834 29216 28855 28983 28852
  28813 29018 28442 287

In [11]:
len(encoded_image)

30403

In [12]:
tmp=[]
tmp.append([])
print(tmp)
tmp[-1].append(3)
print(tmp)

[[]]
[[3]]
